## langchain: il più semplice esempio di Q&A sul contenuto di un documento
Luca Mari, maggio 2023  
[virtenv `langchain`: langchain, openai]

Il problema che si vuole risolvere qui è di usare un LLM per fare domande in linguaggio naturale a proposito del contenuto di uno o più documenti che non sono stati letti durante il pre-training.

Se il documento è sufficientemente breve, tale da essere interamente contenuto in un prompt insieme con la domanda, il processo è molto semplice. Si può infatti passare direttamente il documento all'oggetto `load_qa_chain` come contesto, dunque senza doverlo dividere in chunks e senza dover fare l'embedding dei chunks (d'altra parte, se il documento non è abbastanza breve, l'esecuzione genera un errore, proprio dovuto all'eccessiva lunghezza del prompt inviato all'LLM).

In [1]:
from langchain.document_loaders import TextLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

# leggi il documento, breve
loader = TextLoader('./8tuple_s.txt', encoding='utf8')
data = loader.load()

# crea la domanda
query = "What is the fourth element of the 8-tuple and what is its purpose?"

# attiva l'llm
llm = OpenAI(temperature=0, client='simpleQA')

# invia la domanda senza il contesto, ottieni e visualizza la risposta
print('Senza il contesto:', llm(query).strip())

# invia la domanda con il contesto, ottieni e visualizza la risposta
chain = load_qa_chain(llm, chain_type='stuff')
print('Con il contesto:', chain.run(input_documents=data, question=query).strip())

Senza il contesto: The fourth element of the 8-tuple is the "exception" element, and its purpose is to provide information about any errors that may have occurred during the execution of the program. This element is typically used to store an exception object that contains information about the error, such as the type of error, the line of code where the error occurred, and a description of the error.
Con il contesto: The fourth element of the 8-tuple is the set Ω ⊆ {u: T → U} of the admissible input functions of the model. Its purpose is to specify constraints on the functions that the model accepts as inputs.


Come si vede, senza il contesto, cioè senza la lettura del documento, la risposta è sbagliata, mentre con il contesto, cioè con la lettura del documento, il risultato è di buona qualità.  
È dunque solo se il documento è sufficientemente lungo che occorre dividerlo in chunks, farne l'embedding, e attivare la ricerca semantica per selezionare i chunks rilevanti per la specifica domanda.